In [12]:
# 필수 라이브러리 설치
# pip install yfinance ta pandas matplotlib

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator

# 1. 데이터 불러오기
ticker = "005930.KS"  # 삼성전자
start_date = "2020-01-01"
end_date = "2024-12-31"
df = yf.download(ticker, start=start_date, end=end_date)
df = df.dropna()

# 2. 기술적 지표 계산
close_series = df["Close"]
df["RSI"] = RSIIndicator(close=close_series, window=14).rsi()
df["Volume_Spike"] = df["Volume"] > df["Volume"].rolling(5).mean() * 2
df["Bullish_Candle"] = df["Close"] > df["Open"]

# 3. 공매도 지표 (임의 생성, 실제는 크롤링 필요)
np.random.seed(42)
df["공매도_비중"] = np.random.uniform(0.5, 2.5, len(df))
df["공매도_급증"] = df["공매도_비중"].diff() > 0.5
df["업틱룰_발동"] = np.random.choice([True, False], size=len(df), p=[0.1, 0.9])
df["외인_순매수"] = np.random.randint(-1000, 1000, size=len(df))
df["기관_순매수"] = np.random.randint(-1000, 1000, size=len(df))

# 4. 시그널 생성
df["Signal"] = (
    (df["공매도_비중"] > 1.5) &
    (df["공매도_급증"]) &
    (df["업틱룰_발동"]) &
    (df["RSI"] < 30) &
    (df["Bullish_Candle"]) &
    (df["Volume_Spike"]) &
    (df["외인_순매수"] > 0) &
    (df["기관_순매수"] > 0)
)

# 5. 백테스트
df = df.reset_index()
trades = []

for i in range(len(df) - 6):
    signal = df.at[i, "Signal"]  # <-- 핵심 수정: at 사용 (스칼라 값 보장)
    if pd.notna(signal) and signal == True:
        entry_date = df.at[i + 1, "Date"]
        entry_price = df.at[i + 1, "Open"]
        exit_price = df.at[i + 6, "Close"]
        ret = (exit_price - entry_price) / entry_price
        trades.append({
            "Entry Date": entry_date,
            "Entry Price": entry_price,
            "Exit Price": exit_price,
            "Return": ret
        })

# 6. 결과 출력
results_df = pd.DataFrame(trades)

if not results_df.empty:
    results_df["Cumulative Return"] = (1 + results_df["Return"]).cumprod()
    print(results_df)

    # 7. 그래프
    results_df.plot(x="Entry Date", y="Cumulative Return", marker='o', title="공매도 기반 전략 수익률")
    plt.grid(True)
    plt.show()
else:
    print("📭 신호 조건에 해당하는 매매가 발생하지 않았습니다.")


[*********************100%***********************]  1 of 1 completed


ValueError: Data must be 1-dimensional, got ndarray of shape (1229, 1) instead

In [13]:
%pip install yfinance ta pandas matplotlib

Note: you may need to restart the kernel to use updated packages.
